In [ ]:
pip install torch torchvision torchaudio yolov5 matplotlib shutil

In [ ]:
import os
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from yolov5 import YOLOv5

In [ ]:
pretrained_file_path = '../yolov5m.pt'
model = YOLOv5(pretrained_file_path)

if os.path.exists(pretrained_file_path):
    model = YOLOv5(pretrained_file_path)
    print("Model loaded successfully.")
else:
    raise FileNotFoundError(f"File {pretrained_file_path} not found.")

In [ ]:
# Define source and destination directories
src_dir = '../data/cam1'
dst_dir = '../data/cam1_rgb'

# Create destination directory if it doesn't exist
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Function to delete a directory if it exists
def delete_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)

# Function to recreate a directory
def recreate_directory(directory):
    os.makedirs(directory)

# Function to process images
def process_images(src_dir, dst_dir):
    for filename in os.listdir(src_dir):
        if filename.endswith('.png'):
            # Read the image
            image_path = os.path.join(src_dir, filename)
            image = cv2.imread(image_path)
            
            # Ensure the image is loaded correctly
            if image is not None:
                # Get the right half of the image (640x928)
                height, width = image.shape[:2]
                right_half = image[:, width//2:]

                # Save the right half to the destination directory
                dst_image_path = os.path.join(dst_dir, filename)
                cv2.imwrite(dst_image_path, right_half)
            else:
                print(f"Failed to load image: {image_path}")

# Process the images
process_images(src_dir, dst_dir)

print(f"Processing complete. The right half of the images have been saved to {dst_dir}")

Using rgb values to detect grey reference panel

In [ ]:
import json
import os
from PIL import Image, ImageDraw, ImageOps
import numpy as np

# Define grey RGB values and their tolerance ranges
light_grey_rgb = (160, 160, 1600)
normal_grey_rgb = (60, 60, 60)
dark_grey_rgb = (40, 40, 40)
tolerance = 5  # Tolerance range for grey detection

def is_within_grey_range(pixel_rgb, grey_rgb, tolerance):
    return all(abs(c - g) <= tolerance for c, g in zip(pixel_rgb, grey_rgb))

def find_grey_reference_panel(image, grey_rgb_list, tolerance):
    image_np = np.array(image)
    height, width, _ = image_np.shape
    grey_pixels = []
    
    for y in range(height):
        for x in range(width):
            pixel_rgb = tuple(image_np[y, x])
            if any(is_within_grey_range(pixel_rgb, grey_rgb, tolerance) for grey_rgb in grey_rgb_list):
                grey_pixels.append((x, y))
    
    if not grey_pixels:
        print("No grey reference panel found.")
        return None
    
    # Calculate the average position of detected grey pixels
    avg_x = int(np.mean([x for x, y in grey_pixels]))
    avg_y = int(np.mean([y for x, y in grey_pixels]))
    
    return avg_x, avg_y

def resize_image(image, target_size):
    if image.size == target_size:
        return image
    else:
        return image.resize(target_size)

def segment_image(json_file, image_folder, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(image_folder):
        if not filename.endswith('_1.png'):
            continue
        
        original_img_path = os.path.join(image_folder, filename)
        
        # Check if the original image file exists
        if not os.path.exists(original_img_path):
            print(f"Error: Original image file '{original_img_path}' not found.")
            continue
        
        # Open original image
        original_image = Image.open(original_img_path)
        resized_image = resize_image(original_image, (640, 928))
        
        # Convert image to RGB and find grey reference panel
        resized_image_rgb = resized_image.convert('RGB')
        grey_rgb_list = [light_grey_rgb, dark_grey_rgb, normal_grey_rgb]
        grey_center = find_grey_reference_panel(resized_image_rgb, grey_rgb_list, tolerance)
        
        if grey_center is None:
            continue
        
        # Define size of the rectangle to segment
        segment_size = 30
        avg_x, avg_y = grey_center
        x1 = max(0, avg_x - segment_size // 2)
        y1 = max(0, avg_y - segment_size // 2)
        x2 = min(resized_image.width, x1 + segment_size)
        y2 = min(resized_image.height, y1 + segment_size)
        
        # Create a mask image for the region to be kept
        mask = Image.new('L', resized_image.size, 0)
        draw = ImageDraw.Draw(mask)
        draw.rectangle([x1, y1, x2, y2], fill=255)
        
        # Invert the mask so that the region is white and everything else is black
        inverted_mask = ImageOps.invert(mask)
        
        # Apply the inverted mask to segment the image
        segmented_image = Image.composite(resized_image, Image.new('RGB', resized_image.size, (255, 255, 255)), inverted_mask)
        
        # Save the segmented image with a new filename
        new_filename = f"segmented_{filename}"
        segmented_img_path = os.path.join(output_folder, new_filename)
        segmented_image.save(segmented_img_path)
        print(f"Saved segmented image: {segmented_img_path}")

# Example usage
json_file_path = '../data/cam1.json'
image_folder = '../data/cam1_rgb/'                # Folder containing images
output_folder = '../data/cam1_segmented/'         # Folder to save segmented images

# Perform segmentation
segment_image(json_file_path, image_folder, output_folder)

In [ ]:
import json
import os
from PIL import Image, ImageDraw, ImageOps

def resize_image(image, target_size):
    # Check if the image dimensions match the target size
    if image.size == target_size:
        return image
    else:
        return image.resize(target_size)

def segment_image(json_file, image_folder, output_folder):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    for filename in os.listdir(image_folder):
        if not filename.endswith('_1.png'):
            continue

        # Extract image attributes from JSON data
        image_key = "image_attributes"
        if image_key not in data:
            print(f"No segmentation data found for '{filename}'. Skipping.")
            continue
        
        image_data = data[image_key]
        
        # Construct paths for segmented image and original image
        original_img_path = os.path.join(image_folder, filename)
        
        # Check if the original image file exists
        if not os.path.exists(original_img_path):
            print(f"Error: Original image file '{original_img_path}' not found.")
            continue
        
        # Open original image
        original_image = Image.open(original_img_path)
        
        # Resize image if needed
        target_size = (640, 928)
        resized_image = resize_image(original_image, target_size)
        
        # Create a mask image for the regions to remove
        mask = Image.new('L', resized_image.size, 0)
        draw = ImageDraw.Draw(mask)
        
        regions = image_data.get('regions', [])
        for region in regions:
            region_attributes = region.get('region_attributes', {})
            if region_attributes:
                # If region_attributes is empty, mark for removal
                shape_attributes = region.get('shape_attributes', {})
                x = shape_attributes.get('x', 0)
                y = shape_attributes.get('y', 0)
                width = shape_attributes.get('width', 0)
                height = shape_attributes.get('height', 0)
                
                # Draw rectangle on mask
                draw.rectangle([x, y, x + width, y + height], fill=255)
        
        # Invert the mask to remove everything except the segmented region
        inverted_mask = ImageOps.invert(mask)
        
        # Apply the inverted mask to segment the resized image
        segmented_image = Image.composite(resized_image, Image.new('RGB', resized_image.size, (255, 255, 255)), inverted_mask)
        
        # Save the segmented image with a new filename
        new_filename = f"segmented_{filename}"
        segmented_img_path = os.path.join(output_folder, new_filename)
        segmented_image.save(segmented_img_path)
        print(f"Saved segmented image: {segmented_img_path}")

# Example usage
json_file_path = '../data/cam1.json'
image_folder = '../data/cam1_rgb/'                # Folder containing images
output_folder = '../data/cam1_segmented/'         # Folder to save segmented images

# Perform segmentation
segment_image(json_file_path, image_folder, output_folder)

### After Use, delete the directories

In [ ]:
delete_directory('../data/cam1_rgb')

In [ ]:
delete_directory('../data/cam1_segmented')

In [ ]:
import numpy as np
from PIL import Image, ImageEnhance
from skimage import morphology
from skimage.segmentation import clear_border
import matplotlib.pyplot as plt
from scipy.ndimage import binary_fill_holes

def preprocess_image(image_path):
    # Load the image
    image = Image.open(image_path)
    image_np = np.array(image)
    
    # Extract the 640x928 region (assume the entire image is already in this size for simplicity)
    rgb = image_np[:928, :640]
    
    # Split the RGB channels
    r, g, b = rgb[:, :, 0], rgb[:, :, 1], rgb[:, :, 2]
    
    # Extract a 100x100 square from the top right corner of the RGB image
    sky = rgb[:100, -100:]
    
    # Split the RGB channels of the sky image
    rs, gs, bs = sky[:, :, 0], sky[:, :, 1], sky[:, :, 2]
    
    # Calculate the absolute difference between green and blue channels
    gb_diff = np.abs(g - b)
    
    # Create a binary image by thresholding
    gb_binary = gb_diff < 2
    
    # Remove objects connected to the border of the image
    gb_cleared = morphology.remove_small_objects(gb_binary, min_size=300)
    
    # Fill holes in the binary image
    gb_filled = binary_fill_holes(gb_cleared)
    
    # Convert numpy array back to PIL image
    gb_filled_img = Image.fromarray((gb_filled * 255).astype(np.uint8))
    
    # Show the processed images
    plt.figure()
    plt.subplot(1, 2, 1)
    plt.imshow(gb_diff, cmap='gray')
    plt.title('Green-Blue Difference')

    plt.subplot(1, 2, 2)
    plt.imshow(gb_filled, cmap='gray')
    plt.title('Processed Binary Image')
    plt.show()

    return gb_filled_img

def adjust_brightness(image_path):
    # Load the image
    image = Image.open(image_path)
    image_np = np.array(image)
    
    # Extract a 100x100 square from the top right corner of the image
    sky = image_np[:100, -100:]
    
    # Calculate the average red, green, and blue values
    avg_r = np.mean(sky[:, :, 0])
    avg_g = np.mean(sky[:, :, 1])
    avg_b = np.mean(sky[:, :, 2])
    
    # Calculate the overall brightness factor as the average of the RGB components
    brightness_factor = (avg_r + avg_g + avg_b) / (3 * 255)
    
    # Brighten the image by scaling RGB values
    enhancer = ImageEnhance.Brightness(image)
    brightened_image = enhancer.enhance(brightness_factor)
    
    return brightened_image

In [ ]:
# Example usage
image_path = '../data/cam1_rgb/date_1-5-2024_10.0.11_1.png'
segmented_image = preprocess_image(image_path)

# Save the processed images
segmented_image.save('../data/segmented_image.png')

In [ ]:
brightened_image = adjust_brightness(image_path)

# Save the processed images
brightened_image.save('../data/brightened_image.png')

In [ ]:
# Load the image
image = Image.open('../data/cam1_rgb/date_1-5-2024_10.0.11_1.png')

# Crop the image (left, upper, right, lower)
cropped_image = image.crop((0, 0, 640, 928))

# Save the cropped image
cropped_image.save('../data/cropped_image.png')